TODO:

In [1]:
import pandas as pd
import numpy as np

from tools.ffm_tools import df_to_ffm
from tools.cv_tools import train_test_split
from models.base import create_user, site_app_split

In [2]:
categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                    'user', 'device_conn_type', 'C14','C17','C20','C21']

## Convert test

In [2]:
size='tiny'

df_test = pd.read_csv(f'./data/test_{size}.csv')
df_test = create_user(df_test)

df_test_site, df_test_app = site_app_split(df_test)
df_test_site.shape, df_test_app.shape

((624031, 22), (375969, 22))

In [4]:
df_to_ffm(df_test_site, categorical_features, 
          f'./data/test_site_{size}.ffm', test=True)

/home/atkm/code/avazu-ctr/tools/ffm_tools.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c] = f'{c}_' + df[c].astype('str')


In [5]:
df_to_ffm(df_test_app, categorical_features, 
          f'./data/test_app_{size}.ffm', test=True)

In [6]:
raise

RuntimeError: No active exception to reraise

## Develop splits for cross-validation
Need to develop a cross-validation suite for the libffm format.
Start with last day vs rest.

05m30s for site-train/validate, 03m10s for app-train/validate.

In [ ]:
from tools.cv_tools import train_test_split
from models.base import site_app_split

def make_train_validate_data(df, name):
    test_day = 30
    df_train, df_validate = train_test_split(df, None, test_day)
    df_train_site, df_train_app = site_app_split(df_train)
    df_validate_site, df_validate_app = site_app_split(df_validate)
    
    train_site_out = f'./data/train_site_{name}.ffm'
    validate_site_out = f'./data/validate_site_{name}.ffm'
    train_app_out = f'./data/train_app_{name}.ffm'
    validate_app_out = f'./data/validate_app_{name}.ffm'
    
    print(train_site_out)
    df_to_ffm(df_train_site, categorical_features, 
          train_site_out)
    print(validate_site_out)
    df_to_ffm(df_validate_site, categorical_features, 
          validate_site_out)
    print(train_app_out)
    df_to_ffm(df_train_app, categorical_features, 
          train_app_out)
    print(validate_app_out)
    df_to_ffm(df_validate_app, categorical_features, 
          validate_app_out)

In [ ]:
size='small'

df = pd.read_csv(f'./data/train_{size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

test_day = 30
df_train, df_validate = train_test_split(df, None, test_day)
df_train_site, df_train_app = site_app_split(df_train)
df_validate_site, df_validate_app = site_app_split(df_validate)

list(map(lambda df: df.shape, [df_train_site, df_validate_site, df_train_app, df_validate_app]))

In [ ]:
%%time
df_to_ffm(df_train_site, categorical_features, 
          f'./data/train_site_{size}.ffm')
df_to_ffm(df_validate_site, categorical_features, 
          f'./data/validate_site_{size}.ffm')

In [ ]:
%%time
df_to_ffm(df_train_app, categorical_features, 
          f'./data/train_app_{size}.ffm')
df_to_ffm(df_validate_app, categorical_features, 
          f'./data/validate_app_{size}.ffm')

## Encode while writing to file
04m30s to write site-small, 02m30s for app-small.

In [ ]:
df = pd.read_csv('./data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)
df_site = df_site[categorical_features + ['click']]
df_app = df_app[categorical_features + ['click']]
df_site.head()

In [ ]:
site_field_dict = make_field_dict(df_site, categorical_features)
site_feature_dict = make_feature_dict(df_site, categorical_features)
#df_site_encoded = encode_features(df_site, site_feature_dict, categorical_features)
#df_site_encoded.head()

In [ ]:
app_field_dict = make_field_dict(df_app, categorical_features)
app_feature_dict = make_feature_dict(df_app, categorical_features)
#df_app_encoded = encode_features(df_app, app_feature_dict, categorical_features)

In [ ]:
#df_site_encoded.columns[:-1]

In [ ]:
df_site.head()

In [ ]:
fname = './data/train_site_tiny.ffm'
df_to_ffm(df_site, categorical_features, fname)

In [ ]:
%%time
with open('./data/train_site_tiny.ffm', 'w') as f:
    for ffm_row in ffm_row_generator(df_site, site_feature_dict):
        f.write(ffm_row)
        f.write('\n')

In [ ]:
%%time
with open('./data/train_app_tiny.ffm', 'w') as f:
    for ffm_row in ffm_row_generator(df_app, app_feature_dict):
        f.write(ffm_row)
        f.write('\n')

In [ ]:
raise

## `totoruo/FfmEncoder`
Gets stuck at 'converting data...' at line 353.
Try `Bobe24/Dataframe2libffm`.

In [ ]:
df = pd.read_csv('./data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)

In [ ]:
categorical_features = ['banner_pos', 'platform_id', 'platform_category',
                        'user', 'device_conn_type', 'C14','C17','C20','C21']

In [ ]:
from tools.FfmEncoder import FfmEncoder

In [ ]:
encoder = FfmEncoder(categorical_features, label_name='click', nthread=2)

In [ ]:
encoder.transform(df_app, 'train_app.ffm')

In [ ]:
raise

## LibFFM conversion stats: memory issue
With categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                        'user', 'device_conn_type', 'C14','C17','C20','C21']
- just a few seconds to build feature_dict for app-small.
- 12s for app-mid. len = 0.8M
- 21s for site-mid. len = 1.6M
- encode_features exhausts memory...

In [ ]:
def make_field_dict(df, fields):
    """
    fields: Array[String] - a list of column names
    A field dictionary is just an inverted column index.
    """
    return {col: i for i, col in enumerate(fields)}

def make_feature_dict(df, fields):
    # prepend a field name to each feature in order to distinguish 
    # a feature name present in two or more fields.
    for c in fields:
        df[c] = f'{c}_' + df[c].astype('str')
    # TODO: decide whether to hash all features at this stage.
    # TODO: hash into a smaller space to make the dict smaller
    #df[fields] = df[fields].applymap(hash)
    # Index features from all fields.
    features_concat = pd.concat([df[c] for c in fields], ignore_index=True)
    uniques = features_concat.unique()
    return pd.Series(np.arange(len(uniques)),index=uniques)

def encode_features(df, feature_dict):
    return df.replace(feature_dict)

def encode_features_old(df, feature_dict, fields):
    # df.replace(feature_dict) doesn't fit in memory.
    # optimize by splitting the feature_dict into dictionaries corresponding to fields.
    replace_dict = dict()
    for c in fields:
        replace_dict[c] = {k: v for k, v in feature_dict.items() 
                           if k.startswith(c)}
    
    return df.replace(replace_dict)

In [ ]:
df = pd.read_csv('./data/train_small.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)

In [ ]:
#categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
#                        'user', 'device_conn_type', 'C14','C17','C20','C21']
categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                         'device_conn_type', 'C14','C17','C20','C21']

In [ ]:
field_dict = make_field_dict(df_site, categorical_features)
feature_dict = make_feature_dict(df_site, categorical_features)

In [ ]:
%%time
df_site_encoded = encode_features(df_site, feature_dict)

In [ ]:
len(feature_dict)

In [ ]:
raise

## Develop LibFFM format converter
Refer to the Data Format section of https://github.com/guestwalk/libffm .

In [ ]:
df = pd.read_csv('./data/train_tiny.csv')
categorical_features = ['C1','device_type']
df.head()

In [ ]:
field_dict = make_field_dict(df, categorical_features)
feature_dict = make_feature_dict(df, categorical_features)

In [ ]:
feature_dict

In [ ]:
df_encoded = encode_features(df, feature_dict)

## Quickstart
xl has ffm, fm, and linear models. ffm models only take the libffm format, while others take csv or the libsvm format.

In [ ]:
ffm_model = xl.create_ffm()

In [ ]:
ffm_model.setTrain('./data/small_train.txt')

In [ ]:
# param:
#  0. task: binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', 'lr':0.2, 'lambda':0.002}

In [ ]:
ffm_model.fit(param, "./model.out")

In [ ]:
ffm_model.setTest('./data/small_test.txt')

In [ ]:
ffm_model.predict('./model.out', './xlearn-output.txt')

## xlearn with its sklearn API (deprecated)
FFMModel.fit takes array-like. The format of its fields argument is unclear.

In [ ]:
ffm = xl.FFMModel(task='binary', lr=0.2, epoch=10, reg_lambda=0.002, metric='acc')
#ffm.fit('./data/small_train.txt')